In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
# import kagglehub
import torch
import numpy as np
from transformers import AutoProcessor, AutoModelForImageTextToText
# import cv2
import json
import datetime
import os
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import base64
import io
from PIL import Image
import logging

GEMMA_PATH = "models/gemma-3n-transformers-gemma-3n-e2b-it-v2"

processor = AutoProcessor.from_pretrained(GEMMA_PATH)
model = AutoModelForImageTextToText.from_pretrained(
    GEMMA_PATH,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.15s/it]



In [12]:
image_file_path = "sample-images/images.webp"
analysis_prompt = "What do you see in the image? Provide a detailed description of the objects, their colors, and any notable features."


with open(image_file_path, "rb") as img_file:
    image_base64_string = base64.b64encode(img_file.read()).decode("utf-8")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_base64_string},
            {"type": "text", "text": analysis_prompt}
        ]
    }
]

In [18]:
# Process with Gemma 3n
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, dtype=model.dtype)

input_len = inputs["input_ids"].shape[-1]
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,
    disable_compile=True,
    temperature=0.7,
    do_sample=True
)

analysis_text = processor.batch_decode(
    outputs[:, input_len:],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
)[0]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation failed with error: number of heads in query/key/value should match

Trying with autocast...


/Users/nipasuma/Projects/gemma-3n-challenge/venv/lib/python3.9/site-packages/torch/amp/autocast_mode.py:283: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation failed with error: number of heads in query/key/value should match

Trying with autocast...


/Users/nipasuma/Projects/gemma-3n-challenge/venv/lib/python3.9/site-packages/torch/amp/autocast_mode.py:283: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation with autocast also failed: number of heads in query/key/value should match

Trying manual forward pass...
Manual forward pass failed: number of heads in query/key/value should match
The issue appears to be deep in the model architecture.


In [8]:
analysis_text

"The capital of India is **New Delhi**. \n\nIt's a very significant city, both historically and politically. \n"